In [1]:
import pandas as pd
import numpy as np
import re
import emoji
import nltk
from matplotlib import pyplot as plt
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
import pyLDAvis.gensim_models
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
from HanTa import HanoverTagger as ht


nltk.download('wordnet')
nltk.download("stopwords")
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Topic Modelling

For topic modelling the same data set as for the sentiment analysis is used, which means that tweets not directly containing the hashtag, as well as retweets are removed. 

In [2]:
df = pd.read_csv ('data/tweets/IchBinHanna.csv')

In [3]:
#get the correct data
df['new_date'] = pd.to_datetime(df['created_at']).dt.strftime('%Y-%m-%d %H:%M:%S')
df = df.loc[(df['new_date'] > '2021-06-01 00:00:00') & (df['new_date'] <= '2021-09-30 23:59:59')]
df = df.loc[df['reference_type'] != 'retweeted']
df = df[df['text'].str.contains("#ichbinhanna", case = False)]
df['processed'] = df['text'].astype(str)
df = df.loc[(df['lang'] == 'de') | (df['lang'] == 'en')]

In [4]:
#clean the data (remove URLs, emojis and line breaks)
def rem_url(tweet):
     return " ".join(re.sub("([^0-9A-Za-zßäöü \t])|(\w+:\/\/\S+)", " ", tweet).split())
df['processed'] = df['processed'].apply(rem_url)
def rem_emojis(text):
    emojis = [x for x in text if x in emoji.UNICODE_EMOJI]
    cleaned = ' '.join([str for str in text.split() if not any(i in str for i in emojis)])
    return cleaned
df['processed'] = df['processed'].apply(lambda x: rem_emojis(x))
df['processed'] = df['text'].replace(r'\\n',  ' ', regex=True)
df['processed'] = df['processed'].astype(str)

In [5]:
#preprocessing (tokenization, stop word removal, stemming)
german_stop = set(stopwords.words('german'))
english_stop = set(stopwords.words('english'))
add_stop_all = ["ichbinhanna","#ichbinhanna", "hanna", "mehr", "innen", "#wisszeitvg", "#ichbinhannah", "@amreibahr", "amreibahr", "@bmf_bund","bmf_bund" "@drkeichhorn", "@bmbf_bund", "mehr", "innen", "schon", "gehen", "jahr","wissenschaft", "wissenschaftler", "kommen","academia", "academic", "year", "machen", "sagen", "sein","geben", "also", "werden", "german", "germany","gut", "haben", "geht", "gibt", "viele", "seit", "wäre"]
german_stop.update(set(add_stop_all))
english_stop.update(set(add_stop_all))
tweet_tokenizer = TweetTokenizer()
df['tokenized'] = df['processed'].apply(lambda x: tweet_tokenizer.tokenize(x.lower()))
df['tokenized'] = df[['tokenized','lang']].apply(lambda x: ' '.join([word for word in x['tokenized'] if word not in english_stop]).split() if x['lang'] == 'en' else ' '.join([word for word in x['tokenized'] if word not in german_stop]).split(),axis=1)
#remove all words consisting of 3 characters or less to shorten the dictionary of unique words
df['tokenized'] = df['tokenized'].apply(lambda x: [word for word in x if len(word) > 3])
lemmatizer = WordNetLemmatizer()
hannover = ht.HanoverTagger('morphmodel_ger.pgz')
df['tokenized'] = df[['tokenized','lang']].apply(lambda x: [lemmatizer.lemmatize(word).lower() for word in x['tokenized']] if x['lang'] == 'en' else [hannover.analyze(word)[0].lower() for word in x['tokenized']] ,axis=1)

In [43]:
def perform_LDA(tokens, topics=5, passes =5, alpha = 'symmetric', decay = 0.5):
    #create the dictionary of lemmatized tokens
    dic = Dictionary(tokens)
    print(len(dic))
    #remove low and high frequent terms
    dic.filter_extremes(no_below=2, no_above=.99)
    print(len(dic))
    #create the bag of words 
    corpus = [dic.doc2bow(d) for d in tokens]
    #build LDA model 
    LDA = LdaMulticore(corpus= corpus, num_topics=topics, id2word= dic, workers=12, passes=passes, alpha = alpha, decay = decay)
    words = [re.findall(r'"([^"]*)"',t[1]) for t in LDA.print_topics()]
    #create topics
    topics = [' '.join(t[0:10]) for t in words]

    for id, t in enumerate(topics): 
        print(f"------ Topic {id} ------")
        print(t, end="\n\n")
    # Compute Perplexity
    perplexity = LDA.log_perplexity(corpus)
    print('\nPerplexity: ', perplexity) 
    # Compute Coherence Score
    coherence_model = CoherenceModel(model=LDA, texts=tokens, 
                                   dictionary=dic, coherence='c_v')
    coherence_lda_model = coherence_model.get_coherence()
    print('\nCoherence Score: ', coherence_lda_model)
    return LDA

In [7]:
#perform topic modelling by applying LDA on the whole data set, and on the german/english only sets (guided by: https://towardsdatascience.com/twitter-topic-modeling-e0e3315b12e2)
#full data set
full_model = perform_LDA(df['tokenized'])

47228
15469
------ Topic 0 ------
werden machen problem contract system finden mensch immer scholar ganz

------ Topic 1 ------
jahr gut machen immer arbeitsbedingung forschung werden brauchen prekär finden

------ Topic 2 ------
jahr gut frage befristet #hannaimbundestag wichtig arbeit neu eigentlich zeit

------ Topic 3 ------
jahr arbeit gut befristet problem müssen warum stellen vertrag @anjakarliczek

------ Topic 4 ------
@anjakarliczek system wissen arbeit #ichbinreyhan problem aktuell research @drkeichhorn #hannaimbundestag


Perplexity:  -8.672711893652302

Coherence Score:  0.25304525502446684


In [8]:
#only german tweets
df_ger = df.loc[df['lang'] == "de"]
df_en = df.loc[df['lang'] == "en"]
ger_model = perform_LDA(df_ger['tokenized'])

39811
12601
------ Topic 0 ------
gut jahr machen befristet system arbeit wissen müssen vielen haben

------ Topic 1 ------
wichtig arbeit wissen ganz forschung wissenschaftlich befristet deutschland problem lehre

------ Topic 2 ------
gut @anjakarliczek aktuell @gew_bund eigentlich hochschule arbeit werden stunde jahr

------ Topic 3 ------
jahr vertrag befristet @anjakarliczek heute zeit promotion groß arbeit neu

------ Topic 4 ------
machen stellen jahr immer forschung haben unbefristet @anjakarliczek gerade werden


Perplexity:  -8.477697358283951

Coherence Score:  0.166008948057337


In [9]:
#only english tweets
en_model = perform_LDA(df_en['tokenized'])

9256
3328
------ Topic 0 ------
researcher contract precarious scholar research many condition career #ichbinreyhan discussion

------ Topic 1 ------
system much read need know research student thread people work

------ Topic 2 ------
year system research contract work academic position career postdoc university

------ Topic 3 ------
working condition university system problem time contract like english permanent

------ Topic 4 ------
thread contract story science share position want university people need


Perplexity:  -7.8100907723873645

Coherence Score:  0.22015641867679295


In [10]:
#get topics by month
df_june = df.loc[(df['new_date'] > '2021-06-01 00:00:00') & (df['new_date'] <= '2021-06-30 23:59:59')]
df_july = df.loc[(df['new_date'] > '2021-07-01 00:00:00') & (df['new_date'] <= '2021-07-31 23:59:59')]
df_august = df.loc[(df['new_date'] > '2021-08-01 00:00:00') & (df['new_date'] <= '2021-08-31 23:59:59')]
df_september = df.loc[(df['new_date'] > '2021-09-01 00:00:00') & (df['new_date'] <= '2021-09-30 23:59:59')]

In [11]:
#tweets from june
june_model = perform_LDA(df_june['tokenized'])

31199
10502
------ Topic 0 ------
jahr arbeit befristet wissenschaftlich #hannaimbundestag promotion finden problem stellen forschung

------ Topic 1 ------
system video deutsch #bmbf hochschule ganz gerade #hannaimbundestag statt career

------ Topic 2 ------
jahr forschung befristet machen lehre vertrag arbeit haben gut immer

------ Topic 3 ------
jahr wissen @anjakarliczek #hannaimbundestag gut hashtag immer letzter thread arbeit

------ Topic 4 ------
#hannaimbundestag @anjakarliczek gut befristet stelle müssen arbeit problem machen heute


Perplexity:  -8.492904243419366

Coherence Score:  0.2266602332809593


In [12]:
#tweets from july
july_model = perform_LDA(df_july['tokenized'])

16767
5780
------ Topic 0 ------
arbeit jahr müssen gut @drkeichhorn wissenschaftlich mensch sprechen promotion sollen

------ Topic 1 ------
@gew_bund problem story gerade #ichbinreyhan einfach #dauerstell klar werden gut

------ Topic 2 ------
gut @anjakarliczek immer wissen nachwuchs ganz arbeit system wichtig machen

------ Topic 3 ------
#ichbinreyhan @tagesthemen zeit @anjakarliczek forschung machen befristet stellen danken immer

------ Topic 4 ------
gut #ichbinreyhan jahr brauchen system @anjakarliczek arbeitsbedingung thema @gew_bund diskussion


Perplexity:  -8.39668779293529

Coherence Score:  0.26668444384190404


In [13]:
#tweets from august
august_model = perform_LDA(df_august['tokenized'])

9918
3233
------ Topic 0 ------
arbeit @drkeichhorn #ichbinreyhan @anjakarliczek forschung gut aktuell sprechen brauchen wissen

------ Topic 1 ------
#ichbinreyhan gut immer stellen bleiben machen gerade werden perspektive @drkeichhorn

------ Topic 2 ------
arbeit system @drkeichhorn #wissenschaft gut #ichbinreyhan haben year heute @heutejournal

------ Topic 3 ------
#dauerstell @gew_bund #ichbinreyhan befristet wissen ganz problem finden sollen @akellergew

------ Topic 4 ------
#ichbinreyhan machen system gut @anjakarliczek wichtig stellen müssen ganz hochschule


Perplexity:  -8.065780770680187

Coherence Score:  0.28093361137968315


In [14]:
#tweets from september
september_model = perform_LDA(df_september['tokenized'])

9231
3015
------ Topic 0 ------
#ichbinreyhan @gew_bund hochschule jahr #entfristethanna #dauerstell prekär gut #hannaorganisiertsich arbeit

------ Topic 1 ------
@gew_bund arbeit machen vertrag befristet #ichbinreyhan immer müssen hochschule jahr

------ Topic 2 ------
machen heute immer @andreasbovensc1 #ichbinreyhan ganz @swh_hb sehen gut @spdlandbremen

------ Topic 3 ------
#ichbinreyhan gut monat jahr vertrag wissen arbeit #hannainzahlen @drkeichhorn haben

------ Topic 4 ------
neu stellen #ichbinreyhan gut forschung #btw21 system lehre #berlhg jahr


Perplexity:  -8.020997995103459

Coherence Score:  0.29005676156280746


In [69]:
def visualize_topics(tokens):
    #Visualization of topic distance 
    vis_dic = Dictionary(tokens)
    vis_dic.filter_extremes(no_below=2, no_above=.99)
    #create the bag of words 
    vis_corpus = [vis_dic.doc2bow(d) for d in tokens]
    #build LDA model a 
    vis_LDA = LdaMulticore(corpus= vis_corpus, num_topics=5, id2word= vis_dic, workers=12, passes=5, random_state = 1)
    pyLDAvis.enable_notebook()
    return pyLDAvis.gensim_models.prepare(vis_LDA, vis_corpus, vis_dic)
def visualize_model(model, tokens):
    vis_dic = Dictionary(tokens)
    vis_dic.filter_extremes(no_below=2, no_above=.99)
    vis_corpus = [vis_dic.doc2bow(d) for d in tokens]
    pyLDAvis.enable_notebook()
    return pyLDAvis.gensim_models.prepare(model, vis_corpus, vis_dic)

In [61]:
visualize_topics(df['tokenized'])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.074691  0.008792       1        1  26.107870
3     -0.038716  0.004221       2        1  23.084935
0     -0.014871 -0.006734       3        1  19.648851
1      0.072421  0.064642       4        1  15.867998
4      0.055858 -0.070921       5        1  15.290347, topic_info=          Term         Freq        Total Category  logprob  loglift
150       jahr  1385.000000  1385.000000  Default  30.0000  30.0000
286    problem   671.000000   671.000000  Default  29.0000  29.0000
626    letzter   410.000000   410.000000  Default  28.0000  28.0000
839    vertrag   589.000000   589.000000  Default  27.0000  27.0000
205  befristet   881.000000   881.000000  Default  26.0000  26.0000
..         ...          ...          ...      ...      ...      ...
161     arbeit    90.206879   987.327215   Topic5  -5.9613  -0.5149
168     machen    84.556435   783.247687   Topic5  -6.0259  -0.3481
153  schreiben    72.141524   400.781790   Topic5  -6.1847   0.1632
619      frage    67.548806   436.125820   Topic5  -6.2505   0.0129
930      immer    69.213731   712.443802   Topic5  -6.2262  -0.4536

[489 rows x 6 columns], token_table=       Topic      Freq                      Term
term                                            
480        1  0.179538           #95vswisszeitvg
480        2  0.113937           #95vswisszeitvg
480        3  0.341812           #95vswisszeitvg
480        4  0.262402           #95vswisszeitvg
480        5  0.103580           #95vswisszeitvg
...      ...       ...                       ...
14956      1  0.965568                     ️ende
14957      1  0.874738  ️erwerbsarbeitslosigkeit
14958      1  0.865469                   ️kinder
14960      1  0.888337               ️stipendium
14960      2  0.080758               ️stipendium

[1284 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 1, 2, 5])

In [40]:
#get optimal number of topics for each (sub)set
def compute_coherence_values_topics(tokens, limit=10, start=2, step=1):
    
    dic = Dictionary(tokens)
    dic.filter_extremes(no_below=2, no_above=.99)
    corpus = [dic.doc2bow(d) for d in tokens]
    coherence_values_topic = []
    model_list_topic = []
    for num_topics in range(start, limit, step):
        model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=dic, random_state = 1)
        model_list_topic.append(num_topics)
        coherencemodel = CoherenceModel(model=model, texts=tokens, dictionary=dic, coherence='c_v')
        coherence_values_topic.append(coherencemodel.get_coherence())

    print(model_list_topic, coherence_values_topic)
    
def compute_coherence_values_passes(tokens,num_topics):
    
    passes = [5,10,15,20]
    dic = Dictionary(tokens)
    dic.filter_extremes(no_below=2, no_above=.99)
    corpus = [dic.doc2bow(d) for d in tokens]
    coherence_values_passes = []
    model_list_passes = []
    for num_pass in passes:
        model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=dic, passes = num_pass, random_state = 1)
        model_list_passes.append(num_pass)
        coherencemodel = CoherenceModel(model=model, texts=tokens, dictionary=dic, coherence='c_v')
        coherence_values_passes.append(coherencemodel.get_coherence())

    print(model_list_passes, coherence_values_passes)
    
def compute_coherence_values_alpha(tokens,num_topics, passes):
    
    alpha = ['symmetric','asymmetric']
    dic = Dictionary(tokens)
    dic.filter_extremes(no_below=2, no_above=.99)
    corpus = [dic.doc2bow(d) for d in tokens]
    coherence_values_alpha = []
    model_list_alpha = []
    for a in alpha:
        model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=dic, passes = passes, alpha = a, random_state = 1)
        model_list_alpha.append(a)
        coherencemodel = CoherenceModel(model=model, texts=tokens, dictionary=dic, coherence='c_v')
        coherence_values_alpha.append(coherencemodel.get_coherence())

    print(model_list_alpha, coherence_values_alpha)
    
def compute_coherence_values_decay(tokens,num_topics, passes, alpha):
    
    decay = [0.5,0.7,0.9]
    dic = Dictionary(tokens)
    dic.filter_extremes(no_below=2, no_above=.99)
    corpus = [dic.doc2bow(d) for d in tokens]
    coherence_values_decay = []
    model_list_decay = []
    for d in decay:
        model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=dic, passes = passes, alpha = alpha, random_state = 1, decay =d)
        model_list_decay.append(d)
        coherencemodel = CoherenceModel(model=model, texts=tokens, dictionary=dic, coherence='c_v')
        coherence_values_decay.append(coherencemodel.get_coherence())

    print(model_list_decay, coherence_values_decay)

In [28]:
#full data
compute_coherence_values_topics(df['tokenized'])
#english data
compute_coherence_values_topics(df_en['tokenized'])
#german data
compute_coherence_values_topics(df_ger['tokenized'])
#june data
compute_coherence_values_topics(df_june['tokenized'])
#july data
compute_coherence_values_topics(df_july['tokenized'])
#august data
compute_coherence_values_topics(df_august['tokenized'])
#september data
compute_coherence_values_topics(df_september['tokenized'])

[2, 3, 4, 5, 6, 7, 8, 9] [0.12389419281098238, 0.11705462478800728, 0.12330710455576704, 0.15429370641598833, 0.21100178941858486, 0.16638388526021494, 0.15783920306806049, 0.20062631867865002]
[2, 3, 4, 5, 6, 7, 8, 9] [0.1668900092320914, 0.1692360171278822, 0.19683454198902256, 0.1966770684631832, 0.20727713470776118, 0.22494371825845016, 0.23695433706018854, 0.21506778494477674]
[2, 3, 4, 5, 6, 7, 8, 9] [0.09583580231313711, 0.10958211640962456, 0.1379404450229763, 0.11690489842666756, 0.15478297101880575, 0.15800747727594208, 0.1803250250191899, 0.16931867258229377]
[2, 3, 4, 5, 6, 7, 8, 9] [0.12837922840473015, 0.12517048445142975, 0.14824822735800955, 0.16128090424783265, 0.1900565105234718, 0.19104573664257887, 0.2213348855479249, 0.19995845461644024]
[2, 3, 4, 5, 6, 7, 8, 9] [0.19961240061184116, 0.16329878859792116, 0.2012796133778894, 0.19263729552448028, 0.21258579087603127, 0.22960642961105915, 0.2792235085771986, 0.30649709139589126]
[2, 3, 4, 5, 6, 7, 8, 9] [0.25179550503

In [31]:
#full data
compute_coherence_values_passes(df['tokenized'], 6)
#english data
compute_coherence_values_passes(df_en['tokenized'],8)
#german data
compute_coherence_values_passes(df_ger['tokenized'],8)
#june data
compute_coherence_values_passes(df_june['tokenized'],8)
#july data
compute_coherence_values_passes(df_july['tokenized'],9)
#august data
compute_coherence_values_passes(df_august['tokenized'],9)
#september data
compute_coherence_values_passes(df_september['tokenized'],8)

[5, 10, 15, 20] [0.21769261620423222, 0.2690152610411907, 0.3228316499418125, 0.3268002840348955]
[5, 10, 15, 20] [0.24915991509879745, 0.2612628445116889, 0.26380476019690424, 0.271753624443677]
[5, 10, 15, 20] [0.19513052324024732, 0.21424691226268477, 0.23010446959167213, 0.23881264878405628]
[5, 10, 15, 20] [0.24474511422358788, 0.2820813185439542, 0.32473946611873705, 0.3381229427915676]
[5, 10, 15, 20] [0.2554959019382797, 0.24402870026977952, 0.2534215279594163, 0.2621248722547103]
[5, 10, 15, 20] [0.2824516647033783, 0.29574032006188355, 0.29849572880663794, 0.30912229640979666]
[5, 10, 15, 20] [0.32174582869474205, 0.31718306381680816, 0.3142800230028888, 0.32047759338204074]


In [37]:
#full data
compute_coherence_values_alpha(df['tokenized'], 6,20)
#english data
compute_coherence_values_alpha(df_en['tokenized'],8,20)
#german data
compute_coherence_values_alpha(df_ger['tokenized'],8,20)
#june data
compute_coherence_values_alpha(df_june['tokenized'],8,20)
#july data
compute_coherence_values_alpha(df_july['tokenized'],9,20)
#august data
compute_coherence_values_alpha(df_august['tokenized'],9,20)
#september data
compute_coherence_values_alpha(df_september['tokenized'],8,5)

['symmetric', 'asymmetric'] [0.3268002840348955, 0.37571561994425395]
['symmetric', 'asymmetric'] [0.271753624443677, 0.2538859498789313]
['symmetric', 'asymmetric'] [0.23678522027750526, 0.21145266638112842]
['symmetric', 'asymmetric'] [0.3421015846690872, 0.39217997909674135]
['symmetric', 'asymmetric'] [0.2621248722547103, 0.35178133343358603]
['symmetric', 'asymmetric'] [0.30912229640979666, 0.36407715047510764]
['symmetric', 'asymmetric'] [0.32174582869474205, 0.3279922890145045]


In [41]:
#full data
compute_coherence_values_decay(df['tokenized'], 6,20,'asymmetric')
#english data
compute_coherence_values_decay(df_en['tokenized'],8,20,'symmetric')
#german data
compute_coherence_values_decay(df_ger['tokenized'],8,20,'symmetric')
#june data
compute_coherence_values_decay(df_june['tokenized'],8,20,'asymmetric')
#july data
compute_coherence_values_decay(df_july['tokenized'],9,20,'asymmetric')
#august data
compute_coherence_values_decay(df_august['tokenized'],9,20,'asymmetric')
#september data
compute_coherence_values_decay(df_september['tokenized'],8,5,'asymmetric')

[0.5, 0.7, 0.9] [0.3808379631849212, 0.2963046085250385, 0.2671810731763273]
[0.5, 0.7, 0.9] [0.271753624443677, 0.26643195527369967, 0.24938472551264299]
[0.5, 0.7, 0.9] [0.23701694936258688, 0.21988486126282533, 0.1972238567213]
[0.5, 0.7, 0.9] [0.3916118752563964, 0.3606002405298114, 0.29765623633544497]
[0.5, 0.7, 0.9] [0.35178133343358603, 0.31524882276942967, 0.30781519531133594]
[0.5, 0.7, 0.9] [0.36407715047510764, 0.34814232942355183, 0.3292749688157396]
[0.5, 0.7, 0.9] [0.3279922890145045, 0.3288853538609816, 0.33193391268330086]


In [45]:
#get the optimal models
optimal_full = perform_LDA(df['tokenized'],6,20,'asymmetric',0.5)
optimal_en = perform_LDA(df_en['tokenized'],8,20,'symmetric',0.5)
optimal_ger = perform_LDA(df_ger['tokenized'],8,20,'symmetric',0.5)
optimal_june = perform_LDA(df_june['tokenized'],8,20,'asymmetric',0.5)
optimal_july = perform_LDA(df_july['tokenized'],9,20,'asymmetric',0.5)
optimal_august = perform_LDA(df_august['tokenized'],9,20,'asymmetric',0.5)
optimal_september = perform_LDA(df_september['tokenized'],8,5,'asymmetric',0.9)

47228
15469
------ Topic 0 ------
jahr @anjakarliczek befristet gut forschung immer arbeit werden machen haben

------ Topic 1 ------
system contract research work @gew_bund year position career researcher thread

------ Topic 2 ------
stellen unbefristet prekär befristet müssen arbeit gut system heute promotion

------ Topic 3 ------
gut arbeit jahr ganz forschung sehen wissen stellen brauchen müssen

------ Topic 4 ------
#hannaimbundestag @anjakarliczek stattfinden hochschule wählen gut gerade sagen hoffen studierend

------ Topic 5 ------
machen studium eigentlich jahr @anjakarliczek wissen hochschule #ichbinreyhan immer kennen


Perplexity:  -8.486853137726243

Coherence Score:  0.2846346894950325
9256
3328
------ Topic 0 ------
year career tweet research contract english many like working time

------ Topic 1 ------
contract condition visa precarious working research colleague country academic make

------ Topic 2 ------
#ichbinreyhan researcher permanent many problem position pr

In [70]:
visualize_model(optimal_full, df['tokenized'])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.101167  0.031961       1        1  48.165340
1      0.340211  0.037081       2        1  14.943978
3     -0.104891  0.069024       3        1  14.772889
2     -0.087336  0.057318       4        1  12.668451
5     -0.037502 -0.037032       5        1   5.110602
4     -0.009316 -0.158352       6        1   4.338741, topic_info=                   Term         Freq        Total Category  logprob  loglift
748      @anjakarliczek   916.000000   916.000000  Default  30.0000  30.0000
2103  #hannaimbundestag   690.000000   690.000000  Default  29.0000  29.0000
41                  gut  1137.000000  1137.000000  Default  28.0000  28.0000
168              machen   768.000000   768.000000  Default  27.0000  27.0000
430             stellen   721.000000   721.000000  Default  26.0000  26.0000
...                 ...          ...          ...      ...      ...      ...
165               heute    27.042146   528.488612   Topic6  -5.9063   0.1650
272             wichtig    25.287929   523.833499   Topic6  -5.9734   0.1067
286             problem    25.965576   690.915553   Topic6  -5.9470  -0.1437
60               müssen    24.035660   608.769253   Topic6  -6.0242  -0.0943
205           befristet    22.969060   853.323843   Topic6  -6.0696  -0.4774

[483 rows x 6 columns], token_table=       Topic      Freq                      Term
term                                            
12471      3  0.885007             #allesistdrin
3246       1  0.222292                  #btw2021
3246       2  0.044458                  #btw2021
3246       4  0.044458                  #btw2021
3246       5  0.088917                  #btw2021
...      ...       ...                       ...
830        5  0.077674                    ändern
830        6  0.032705                    ändern
1067       1  0.943788            öffentlichkeit
1067       6  0.025508            öffentlichkeit
14957      3  0.865725  ️erwerbsarbeitslosigkeit

[1213 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 4, 3, 6, 5])

In [71]:
visualize_model(optimal_en, df_en['tokenized'])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.026980 -0.018805       1        1  15.875585
3      0.025522 -0.006001       2        1  13.596129
0      0.033994  0.021942       3        1  13.478991
6     -0.018454  0.000228       4        1  13.025912
7      0.082167  0.120664       5        1  12.534252
1      0.015343 -0.020351       6        1  11.747405
4     -0.180793  0.041802       7        1  11.126649
2      0.015243 -0.139477       8        1   8.615078, topic_info=               Term        Freq       Total Category  logprob  loglift
69           thread  152.000000  152.000000  Default  30.0000  30.0000
60        condition  132.000000  132.000000  Default  29.0000  29.0000
599            visa   47.000000   47.000000  Default  28.0000  28.0000
1293   @mahaelhissy   72.000000   72.000000  Default  27.0000  27.0000
28    #ichbinreyhan  111.000000  111.000000  Default  26.0000  26.0000
...             ...         ...         ...      ...      ...      ...
138          system   11.866188  238.941422   Topic8  -5.3699  -0.5509
10             work   10.719390  163.619837   Topic8  -5.4715  -0.2738
410        teaching    9.000011   66.646546   Topic8  -5.6464   0.4495
176           leave    8.609290   50.383536   Topic8  -5.6908   0.6848
583            want    8.622174   90.977087   Topic8  -5.6893   0.0954

[635 rows x 6 columns], token_table=      Topic      Freq              Term
term                                   
671       6  0.834866         #academic
191       1  0.063981  #academictwitter
191       2  0.095971  #academictwitter
191       3  0.095971  #academictwitter
191       4  0.159952  #academictwitter
...     ...       ...               ...
58        7  0.092347              year
58        8  0.049252              year
189       1  0.111109         yesterday
189       2  0.666653         yesterday
189       6  0.222218         yesterday

[1540 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 4, 1, 7, 8, 2, 5, 3])

In [72]:
visualize_model(optimal_ger, df_ger['tokenized'])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.029423 -0.011268       1        1  16.427198
3     -0.049626  0.036230       2        1  13.747548
0     -0.056056 -0.001043       3        1  12.900044
6     -0.008266  0.003176       4        1  12.841127
1      0.106966  0.027737       5        1  11.103400
2      0.078678 -0.119353       6        1  11.096752
7     -0.085942 -0.038887       7        1  11.061059
5      0.043669  0.103407       8        1  10.822873, topic_info=               Term         Freq        Total Category  logprob  loglift
1323        hashtag   276.000000   276.000000  Default  30.0000  30.0000
122            jahr  1337.000000  1337.000000  Default  29.0000  29.0000
572         letzter   380.000000   380.000000  Default  28.0000  28.0000
192   #ichbinreyhan   516.000000   516.000000  Default  27.0000  27.0000
851           immer   704.000000   704.000000  Default  26.0000  26.0000
...             ...          ...          ...      ...      ...      ...
446          finden    65.063255   569.492672   Topic8  -5.7970   0.0541
81           werden    61.047246   623.747927   Topic8  -5.8607  -0.1006
481          wissen    58.213301   673.886603   Topic8  -5.9083  -0.2254
137           heute    55.451727   526.646765   Topic8  -5.9569  -0.0275
122            jahr    55.990925  1337.532763   Topic8  -5.9472  -0.9499

[669 rows x 6 columns], token_table=       Topic      Freq                      Term
term                                            
438        1  0.084208           #95vswisszeitvg
438        2  0.103346           #95vswisszeitvg
438        3  0.214347           #95vswisszeitvg
438        4  0.034449           #95vswisszeitvg
438        5  0.107174           #95vswisszeitvg
...      ...       ...                       ...
12136      3  0.886098        ️arbeitsortwechsel
12129      3  0.921058          ️arbeitsverträge
12131      3  0.899231                     ️ende
12132      3  0.850151  ️erwerbsarbeitslosigkeit
12135      3  0.885706               ️stipendium

[2016 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 1, 7, 2, 3, 8, 6])

In [73]:
visualize_model(optimal_june, df_june['tokenized'])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.134723  0.008070       1        1  44.077829
4     -0.164699  0.115023       2        1  15.451059
1     -0.075374 -0.036601       3        1  11.817026
3     -0.094043  0.013102       4        1   7.206653
6      0.306850  0.123336       5        1   6.902701
2      0.049157 -0.137542       6        1   6.557222
5      0.170511 -0.076711       7        1   4.033668
7     -0.057678 -0.008676       8        1   3.953842, topic_info=                   Term        Freq       Total Category  logprob  loglift
303                jahr  984.000000  984.000000  Default  30.0000  30.0000
341   #hannaimbundestag  671.000000  671.000000  Default  29.0000  29.0000
376               immer  395.000000  395.000000  Default  28.0000  28.0000
1111            vertrag  383.000000  383.000000  Default  27.0000  27.0000
550              stelle  345.000000  345.000000  Default  26.0000  26.0000
...                 ...         ...         ...      ...      ...      ...
723           schreiben   13.030112  257.023936   Topic8  -5.9773   0.2486
303                jahr   14.830501  984.383354   Topic8  -5.8479  -0.9648
1362              bitte   11.706716  159.504302   Topic8  -6.0844   0.6186
537              dürfen   11.700120  169.525121   Topic8  -6.0850   0.5571
574           mittelbau   11.588768  171.883458   Topic8  -6.0945   0.5337

[620 rows x 6 columns], token_table=      Topic      Freq             Term
term                                  
168       1  0.367654  #95vswisszeitvg
168       2  0.060436  #95vswisszeitvg
168       3  0.156127  #95vswisszeitvg
168       4  0.201454  #95vswisszeitvg
168       5  0.025182  #95vswisszeitvg
...     ...       ...              ...
4026      2  0.896965         zugesagt
7715      2  0.871716         zulässig
6936      2  0.872122         zweifeln
2998      1  0.908953       österreich
2998      4  0.058642       österreich

[1611 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 5, 2, 4, 7, 3, 6, 8])

In [74]:
visualize_model(optimal_july, df_july['tokenized'])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.120729  0.086187       1        1  28.136313
1      0.030054 -0.019740       2        1  15.772352
3      0.090227 -0.044801       3        1  13.055261
2     -0.000175  0.154737       4        1  10.959030
4      0.050660 -0.051773       5        1   8.897585
6      0.076213 -0.022977       6        1   6.855802
5      0.023237  0.018468       7        1   6.794829
7      0.010328 -0.001065       8        1   5.308248
8     -0.159816 -0.119035       9        1   4.220582, topic_info=                Term        Freq       Total Category  logprob  loglift
89      @drkeichhorn  146.000000  146.000000  Default  30.0000  30.0000
2412    @tagesthemen  103.000000  103.000000  Default  29.0000  29.0000
315        @gew_bund  135.000000  135.000000  Default  28.0000  28.0000
136             jahr  193.000000  193.000000  Default  27.0000  27.0000
38               gut  247.000000  247.000000  Default  26.0000  26.0000
...              ...         ...         ...      ...      ...      ...
386          einfach    7.940056  111.678327   Topic9  -5.5196   0.5215
164    #ichbinreyhan    8.514017  246.847335   Topic9  -5.4498  -0.2019
147           arbeit    7.971526  182.893644   Topic9  -5.5157   0.0322
1163  @anjakarliczek    7.864524  193.442166   Topic9  -5.5292  -0.0374
99            werden    7.686861  129.455387   Topic9  -5.5520   0.3414

[682 rows x 6 columns], token_table=      Topic      Freq                          Term
term                                               
146       1  0.405885                 #95wisspausen
146       4  0.081177                 #95wisspausen
146       7  0.487062                 #95wisspausen
455       1  0.488483  #acertaindegreeofflexibility
455       4  0.465222  #acertaindegreeofflexibility
...     ...       ...                           ...
785       3  0.049767                      übrigens
785       4  0.049767                      übrigens
785       6  0.024883                      übrigens
785       7  0.049767                      übrigens
785       8  0.049767                      übrigens

[1754 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 4, 3, 5, 7, 6, 8, 9])

In [75]:
visualize_model(optimal_august, df_august['tokenized'])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.081436  0.020479       1        1  39.435312
1     -0.265838  0.001177       2        1  12.216897
2      0.059459  0.091532       3        1  12.149667
3      0.059441  0.091262       4        1   9.377408
4      0.051812  0.013349       5        1   8.193870
5      0.062118 -0.194995       6        1   6.789799
6      0.005466 -0.024422       7        1   4.631295
7     -0.047214  0.016202       8        1   3.691399
8     -0.006679 -0.014584       9        1   3.514352, topic_info=              Term        Freq       Total Category  logprob  loglift
229        aktuell   48.000000   48.000000  Default  30.0000  30.0000
52       forschung   67.000000   67.000000  Default  29.0000  29.0000
39     #dauerstell   43.000000   43.000000  Default  28.0000  28.0000
15   #ichbinreyhan  140.000000  140.000000  Default  27.0000  27.0000
97        sprechen   43.000000   43.000000  Default  26.0000  26.0000
..             ...         ...         ...      ...      ...      ...
342          frage    4.138072   66.826779   Topic9  -5.1707   0.5664
519        stellen    4.041401   66.643377   Topic9  -5.1943   0.5456
72             uni    3.249338   33.684239   Topic9  -5.4125   1.0097
984           bmbf    2.730261   13.553200   Topic9  -5.5865   1.7461
425           zeit    3.025361   68.688560   Topic9  -5.4839   0.2258

[588 rows x 6 columns], token_table=      Topic      Freq                          Term
term                                               
143       8  0.816399         #academicmentalhealth
1493      3  0.821530  #acertaindegreeofflexibility
1761      4  0.878156                          #afd
2336      3  0.869264                 #arbeiterkind
1763      4  0.878178                     #baerbock
...     ...       ...                           ...
239       3  0.095853                    übernehmen
239       4  0.095853                    übernehmen
239       5  0.479267                    übernehmen
87        5  0.598086                  überraschend
87        6  0.199362                  überraschend

[1267 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9])

In [76]:
visualize_model(optimal_september, df_september['tokenized'])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.000120  0.003948       1        1  32.496004
1      0.007724  0.023055       2        1  19.241411
2     -0.026623  0.026056       3        1  11.270239
3     -0.048801  0.120632       4        1   9.289951
6     -0.133344 -0.103127       5        1   8.199617
4      0.004564 -0.013977       6        1   7.917044
5      0.117010 -0.074228       7        1   7.433414
7      0.079351  0.017640       8        1   4.152319, topic_info=                Term       Freq      Total Category  logprob  loglift
51           vertrag  70.000000  70.000000  Default  30.0000  30.0000
186            monat  54.000000  54.000000  Default  29.0000  29.0000
192        @gew_bund  94.000000  94.000000  Default  28.0000  28.0000
1613  #hannainzahlen  47.000000  47.000000  Default  27.0000  27.0000
93               gut  97.000000  97.000000  Default  26.0000  26.0000
...              ...        ...        ...      ...      ...      ...
525           finden   2.459981  36.936715   Topic8  -5.7374   0.4725
185           arbeit   2.635588  82.823711   Topic8  -5.6685  -0.2661
405          postdoc   2.343805  23.837283   Topic8  -5.7858   0.8620
130       hochschule   2.441089  72.909829   Topic8  -5.7451  -0.2153
192        @gew_bund   2.409268  94.678919   Topic8  -5.7582  -0.4897

[569 rows x 6 columns], token_table=      Topic      Freq             Term
term                                  
77        1  0.187441  #95vswisszeitvg
77        2  0.337394  #95vswisszeitvg
77        3  0.074976  #95vswisszeitvg
77        4  0.224929  #95vswisszeitvg
77        5  0.037488  #95vswisszeitvg
...     ...       ...              ...
2412      5  0.632673          ️kinder
2740      5  0.764071         ️mentale
2414      1  0.095465      ️stipendium
2414      4  0.190930      ️stipendium
2414      5  0.668255      ️stipendium

[1378 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 7, 5, 6, 8])